In [ ]:
!pip install transformers==4.50.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.3.4
    Uninstalling huggingface_hub-1.3.4:
      Successfully uninstalled huggingface_hub-1.3.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


In [2]:
# from transformers import pipeline
# qg_model = pipeline("text2text-generation", model="google/flan-t5-base", max_length=128)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
# from sentence_transformers import SentenceTransformer

# Hybrig RAG MODEL FOR Flan-T5-base

In [5]:
!pip install wikipedia-api beautifulsoup4 nltk tiktoken
!pip install faiss-cpu
!pip install rank_bm25
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.9.0-py3-none-any.whl size=15422 sha256=ba95a771e092304e5c5d9fe86e4fe2d0a076830e133bc6ac1f38d8d2a30d292f
  Stored in directory: /root/.cache/pip/wheels/08/22/bd/5181c75f59d48538eb0c0f3246ac541b8a3f0bce3bfd097047
Successfully built wikipedia-api
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 66.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6df722ca86390cda57080bf357795f3ebc8baef63c581d51b5eb0a0466ca5829
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [6]:
import wikipediaapi
import re
import time

wiki = wikipediaapi.Wikipedia(
    language="en",
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent="HybridRAG-POC/1.0 (academic demo)"
)

def fetch_valid_article(title, min_words=200):
    time.sleep(0.5)  # rate limiting

    page = wiki.page(title)
    if not page.exists():
        return None

    text = re.sub(r'\s+', ' ', page.text).strip()
    if len(text.split()) < min_words:
        return None

    return {
        "url": page.fullurl,
        "title": page.title,
        "text": text
    }

In [7]:
import json
import random

# SEED_TITLES = [
#     "Artificial intelligence", "World War II", "Quantum mechanics",
#     "Indian cuisine", "Climate change", "Machine learning",
#     "Blockchain", "Ancient Rome", "Solar energy", "Neural networks"
# ]

# fixed_articles = []
# visited = set()

# while len(fixed_articles) < 200:
#     seed = random.choice(SEED_TITLES)
#     page = wiki.page(seed)

#     for link in page.links.values():
#         if link.title in visited:
#             continue
#         visited.add(link.title)

#         article = fetch_valid_article(link.title)
#         if article:
#             fixed_articles.append(article)
#         if len(fixed_articles) == 200:
#             break

# # Save ONLY URLs (requirement)
# with open("fixed_urls.json", "w") as f:
#     json.dump([a["url"] for a in fixed_articles], f, indent=2)

In [8]:
import requests

HEADERS = {
    "User-Agent": "HybridRAG-POC/1.0 (academic demo)"
}

def get_random_wikipedia_title():
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "random",
        "rnnamespace": 0,
        "rnlimit": 1
    }

    try:
        response = requests.get(
            url,
            params=params,
            headers=HEADERS,
            timeout=10
        )
        response.raise_for_status()

        data = response.json()
        return data["query"]["random"][0]["title"]

    except requests.exceptions.RequestException as e:
        print(f"HTTP error fetching random title: {e}")
        return None
    except (KeyError, ValueError) as e:
        print(f"Parsing error fetching random title: {e}")
        return None

In [9]:
def get_random_articles(n=300, max_attempts=8000):
    articles = []
    visited = set()
    attempts = 0

    while len(articles) < n and attempts < max_attempts:
        attempts += 1
        time.sleep(0.5)  # keep this slow & safe

        title = get_random_wikipedia_title()
        if not title or title in visited:
            continue

        visited.add(title)

        article = fetch_valid_article(title)
        if article:
            articles.append(article)

        if attempts % 200 == 0:
            print(f"Attempts: {attempts}, Collected: {len(articles)}")

    if len(articles) < n:
        raise RuntimeError(
            f"Only collected {len(articles)} articles after {attempts} attempts"
        )

    return articles

In [10]:
# Load fixed URLs
with open("fixed_urls.json") as f:
    fixed_urls = json.load(f)

fixed_articles = []

for i, url in enumerate(fixed_urls, start=1):
    title = url.split("/wiki/")[-1]
    article = fetch_valid_article(title)
    if article:
        fixed_articles.append(article)

    if i % 20 == 0:
        print(f"Loaded {i}/{len(fixed_urls)} fixed articles")

print("Fixed articles fetched:", len(fixed_articles))

Loaded 20/200 fixed articles
Loaded 40/200 fixed articles
Loaded 60/200 fixed articles
Loaded 80/200 fixed articles
Loaded 100/200 fixed articles
Loaded 120/200 fixed articles
Loaded 140/200 fixed articles
Loaded 160/200 fixed articles
Loaded 180/200 fixed articles
Loaded 200/200 fixed articles
Fixed articles fetched: 197


In [11]:
# Fetch random articles
random_articles = get_random_articles(300)

# Combine
corpus = fixed_articles + random_articles
print("Total corpus size:", len(corpus))

Attempts: 200, Collected: 108
Attempts: 400, Collected: 220
Total corpus size: 497


In [12]:
import tiktoken
import uuid

tokenizer = tiktoken.get_encoding("cl100k_base")

def chunk_text(text, min_tokens=200, max_tokens=400, overlap=50):
    tokens = tokenizer.encode(text)
    chunks = []

    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]

        if len(chunk_tokens) >= min_tokens:
            chunks.append(tokenizer.decode(chunk_tokens))

        start += max_tokens - overlap

    return chunks

In [13]:
processed_chunks = []

for article in corpus:
    chunks = chunk_text(article["text"])

    for chunk in chunks:
        processed_chunks.append({
            "chunk_id": str(uuid.uuid4()),
            "url": article["url"],
            "title": article["title"],
            "text": chunk
        })
print("Total chunks created:", len(processed_chunks))

Total chunks created: 3105


In [14]:
with open("wikipedia_corpus_chunks.json", "w") as f:
    json.dump(processed_chunks, f, indent=2)

In [15]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load chunks if needed
texts = [c["text"] for c in processed_chunks]

embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True
)

# Normalize embeddings before indexing
embeddings_normalized = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
dim = embeddings_normalized.shape[1]

# Use IndexFlatIP for cosine similarity AFTER normalization
faiss_index = faiss.IndexFlatIP(dim)
faiss_index.add(embeddings_normalized.astype("float32"))

print("FAISS index size:", faiss_index.ntotal)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/98 [00:00<?, ?it/s]

FAISS index size: 3105


In [16]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab') # Explicitly download punkt_tab as suggested by the error

stop_words = set(stopwords.words("english"))

def preprocess_for_bm25(text):
    tokens = word_tokenize(text.lower())
    return [
        t for t in tokens
        if t not in stop_words
        and t not in string.punctuation
        and len(t) > 1
    ]

tokenized_texts = [preprocess_for_bm25(c["text"]) for c in processed_chunks]
bm25 = BM25Okapi(tokenized_texts)

print("BM25 index ready")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


BM25 index ready


In [17]:
def reciprocal_rank_fusion(dense_ids, sparse_ids, k=60, top_n=5):
    """
    dense_ids: list of chunk indices from dense retrieval
    sparse_ids: list of chunk indices from sparse retrieval
    """

    rrf_scores = {}

    for rank, idx in enumerate(dense_ids):
        rrf_scores[idx] = rrf_scores.get(idx, 0) + 1 / (k + rank + 1)

    for rank, idx in enumerate(sparse_ids):
        rrf_scores[idx] = rrf_scores.get(idx, 0) + 1 / (k + rank + 1)

    ranked = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    return [idx for idx, _ in ranked[:top_n]]

In [18]:
def retrieve_hybrid(query, top_k=5):
    # Dense
    q_emb = embedder.encode([query], convert_to_numpy=True)
    q_emb /= np.linalg.norm(q_emb, axis=1, keepdims=True)
    _, dense_ids = faiss_index.search(q_emb.astype("float32"), top_k)

    # Sparse
    sparse_scores = bm25.get_scores(query.split())
    sparse_ids = np.argsort(sparse_scores)[::-1][:top_k]

    # RRF (USE FUNCTION)
    fused_ids = reciprocal_rank_fusion(
        dense_ids[0],
        sparse_ids,
        k=60,
        top_n=top_k * 2
    )

    # URL deduplication
    seen_urls = set()
    final_chunks = []

    for idx in fused_ids:
        chunk = processed_chunks[idx]
        if chunk["url"] not in seen_urls:
            final_chunks.append(chunk)
            seen_urls.add(chunk["url"])
        if len(final_chunks) == top_k:
            break

    return final_chunks

In [27]:
def build_context(chunks, max_tokens=1024):
    context_parts = []
    total_tokens = 0

    for i, c in enumerate(chunks, 1):
        if "text" not in c or not c["text"]:
            continue

        text = f"[Source {i}] {c['text']}"
        tokens = len(word_tokenize(text))

        if total_tokens + tokens > max_tokens:
            break

        context_parts.append(text)
        total_tokens += tokens

        # print(f"Context tokens: {total_tokens}")
        # print(f"Context text: {text}")

    return "\n".join(context_parts)


In [23]:
from transformers import pipeline

qa = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256
)

def generate_answer(query):
    contexts = retrieve_hybrid(query)
    # context_text = "\n".join([c["text"] for c in contexts])
    context_text = build_context(contexts)

    prompt = f"""
    Answer the question using only the context below.

    Context:
    {context_text}

    Question:
    {query}
    """

    return qa(prompt)[0]["generated_text"]


Device set to use cpu


In [21]:
results = retrieve_hybrid("What is artificial intelligence?")

for r in results:
    print(r["title"], "|", r["url"])
    print(r["text"][:500])

Artificial intelligence | https://en.wikipedia.org/wiki/Artificial_intelligence
 idea that our understanding of human subjectivity is altered by technology created with artificial intelligence. See also Artificial consciousness – Field in cognitive science Artificial intelligence and elections – Impact of AI on political elections Artificial intelligence content detection – Software to detect AI-generated content Artificial intelligence in Wikimedia projects – Use of artificial intelligence to develop Wikipedia and other Wikimedia projects Association for the Advancement of
Computer science | https://en.wikipedia.org/wiki/Computer_science
. Software engineering deals with the organizing and analyzing of software—it does not just deal with the creation or manufacture of new software, but its internal arrangement and maintenance. For example software testing, systems engineering, technical debt and software development processes. Artificial intelligence Artificial intelligence (AI) aims 

In [31]:
print(generate_answer("What are the responsibilities of a database administrator?"))

Installation, configuration and upgrading of Database server software and related products.


In [25]:
results = retrieve_hybrid("What technologies are commonly used in automated decision-making systems?")

for r in results:
    print(r["title"], "|", r["url"])

Automation | https://en.wikipedia.org/wiki/Automation
Artificial intelligence | https://en.wikipedia.org/wiki/Artificial_intelligence
Database administration | https://en.wikipedia.org/wiki/Database_administration


In [29]:
print(generate_answer("What technologies are commonly used in automated decision-making systems?"))

ANN – Artificial neural network DCS – Distributed control system HMI – Human machine interface RPA – Robotic process automation SCADA – Supervisory control and data acquisition PLC – Programmable logic controller


# PART 2

# Question Generation (Automated)

In [95]:
from transformers import pipeline

qg_model = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=1024
)

def generate_answer1(query):
    contexts = retrieve_hybrid(query)
    # context_text = "\n".join([c["text"] for c in contexts])
    context_text = build_context(contexts)

    prompt = f"""
    Answer the question using only the context below.

    Context:
    {context_text}

    Question:
    {query}
    """

    return qg_model(prompt)[0]["generated_text"], context_text


Device set to use cpu


In [33]:
QUESTION_PROMPTS = {
    "factual": "Generate one factual question answered by the text below.",
    "comparative": "Generate one comparative question based on the text below.",
    "inferential": "Generate one inferential 'why' or 'how' question based on the text below.",
    "multi-hop": "Generate one multi-hop question that connects multiple ideas in the text below."
}


In [34]:
from collections import defaultdict

def group_chunks_by_url(chunks):
    grouped = defaultdict(list)
    for c in chunks:
        grouped[c["url"]].append(c)
    return grouped

chunks_by_url = group_chunks_by_url(processed_chunks)


In [35]:
import uuid
import random

def generate_questions_from_corpus(
    chunks_by_url,
    max_questions=100
):
    questions = []
    question_count = 0

    urls = list(chunks_by_url.keys())
    random.shuffle(urls)

    for url in urls:
        chunks = chunks_by_url[url]
        title = chunks[0]["title"]

        # Use first chunk as representative (intro-like)
        context = chunks[0]["text"][:1000]

        for q_type, prompt in QUESTION_PROMPTS.items():
            if question_count >= max_questions:
                return questions

            full_prompt = f"""
{prompt}

Text:
{context}
"""

            try:
                output = qg_model(full_prompt)[0]["generated_text"].strip()
            except Exception:
                continue

            if len(output) < 10:
                continue

            questions.append({
                "question_id": str(uuid.uuid4()),
                "question": output,
                "answer": None,  # will fill next
                "source_url": url,
                "source_title": title,
                "question_type": q_type
            })

            question_count += 1

    return questions


In [36]:
generated_questions = generate_questions_from_corpus(
    chunks_by_url,
    max_questions=100
)

len(generated_questions)


100

In [37]:
def generate_ground_truth_answers(questions, chunks_by_url):
    for q in questions:
        chunks = chunks_by_url[q["source_url"]]
        context = chunks[0]["text"][:1200]

        prompt = f"""
Answer the question using ONLY the text below.

Text:
{context}

Question:
{q["question"]}
"""

        try:
            answer = qg_model(prompt)[0]["generated_text"].strip()
        except Exception:
            answer = ""

        q["answer"] = answer

    return questions


In [38]:
qa_dataset = generate_ground_truth_answers(
    generated_questions,
    chunks_by_url
)


In [39]:
with open("qa_dataset_100.json", "w") as f:
    json.dump(qa_dataset, f, indent=2)


# Evaluation Metrics

Implement MRR (URL-level)

In [40]:
def get_ranked_urls(query, top_k=10):
    results = retrieve_hybrid(query, top_k=top_k)
    return [r["url"] for r in results]


In [41]:
def reciprocal_rank(ranked_urls, ground_truth_url):
    for rank, url in enumerate(ranked_urls, start=1):
        if url == ground_truth_url:
            return 1.0 / rank
    return 0.0


In [42]:
def compute_mrr(qa_dataset, top_k=10):
    rr_scores = []

    for qa in qa_dataset:
        ranked_urls = get_ranked_urls(qa["question"], top_k=top_k)
        rr = reciprocal_rank(ranked_urls, qa["source_url"])
        rr_scores.append(rr)

    return sum(rr_scores) / len(rr_scores)


In [43]:
mrr_score = compute_mrr(qa_dataset, top_k=10)
print(f"MRR@10 (URL-level): {mrr_score:.4f}")


MRR@10 (URL-level): 0.8087


# Custom Metric #1 — Recall@K (URL-level)

In [44]:
def recall_at_k(qa_dataset, k=10):
    hits = 0

    for qa in qa_dataset:
        ranked_urls = get_ranked_urls(qa["question"], top_k=k)
        if qa["source_url"] in ranked_urls:
            hits += 1

    return hits / len(qa_dataset)


In [45]:
recall_10 = recall_at_k(qa_dataset, k=10)
print(f"Recall@10 (URL-level): {recall_10:.4f}")


Recall@10 (URL-level): 0.8900


# Custom Metric #2 — Precision@K (URL-level)

In [46]:
def precision_at_k(qa_dataset, k=10):
    precisions = []

    for qa in qa_dataset:
        ranked_urls = get_ranked_urls(qa["question"], top_k=k)
        precision = 1.0 / k if qa["source_url"] in ranked_urls else 0.0
        precisions.append(precision)

    return sum(precisions) / len(precisions)


In [48]:
precision_5 = precision_at_k(qa_dataset, k=5)
print(f"Precision@5 (URL-level): {precision_5:.4f}")


Precision@5 (URL-level): 0.1760


# Full Evaluation Pipeline

In [61]:
def evaluate_system(qa_dataset):
    results = {
        "MRR@10": compute_mrr(qa_dataset, top_k=10),
        "Recall@10": recall_at_k(qa_dataset, k=10),
        "Precision@5": precision_at_k(qa_dataset, k=5)
    }
    return results


In [50]:
evaluation_results = evaluate_system(qa_dataset)

for metric, value in evaluation_results.items():
    print(f"{metric}: {value:.4f}")


MRR@10: 0.8087
Recall@10: 0.8900
Precision@5: 0.1760


In [62]:
import pandas as pd

results = evaluate_system(qa_dataset)

df = pd.DataFrame(results.items(), columns=["Metric", "Score"])
df

,Metric,Score
0,MRR@10,0.80875
1,Recall@10,0.89000
2,Precision@5,0.17600


In [64]:
import json

results = evaluate_system(qa_dataset)

with open("evaluation_results.json", "w") as f:
    json.dump(results, f, indent=2)


MRR → measures how early the correct document appears

Recall@K → measures whether it appears at all

Precision@K → measures how efficiently it appears in a limited context window

Together, these metrics provide a complete and complementary evaluation of retrieval performance.

The hybrid retrieval system achieves an MRR@10 of 0.8087, indicating that the correct Wikipedia source is typically ranked within the top two results. Recall@10 of 0.8900 shows that the retriever successfully surfaces the correct document for the majority of queries. Precision@5 approaches the theoretical maximum for a single-relevant-document setting, demonstrating efficient and focused retrieval under limited context budgets.

In [103]:
import re
import string

def normalize_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [104]:
def exact_match(pred, gold):
    return int(normalize_text(pred) == normalize_text(gold))


In [105]:
from collections import Counter

def f1_score(pred, gold):
    pred_tokens = normalize_text(pred).split()
    gold_tokens = normalize_text(gold).split()

    common = Counter(pred_tokens) & Counter(gold_tokens)
    num_same = sum(common.values())

    if len(pred_tokens) == 0 or len(gold_tokens) == 0:
        return 0.0
    if num_same == 0:
        return 0.0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)

    return 2 * precision * recall / (precision + recall)


In [113]:
def evaluate_answer_quality(qa_dataset):
    em_scores = []
    f1_scores = []

    for qa_item in qa_dataset:
        pred, _ = generate_answer1(qa_item["question"])
        gold = qa_item["answer"]

        em_scores.append(exact_match(pred, gold))
        f1_scores.append(f1_score(pred, gold))

    return {
        "Exact Match": sum(em_scores) / len(em_scores),
        "F1": sum(f1_scores) / len(f1_scores)
    }

In [107]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


In [108]:
def rouge_l_score(pred, gold):
    score = scorer.score(gold, pred)
    return score["rougeL"].fmeasure


In [114]:
def evaluate_rouge_l(qa_dataset):
    scores = []

    for qa_item in qa_dataset:
        pred, _ = generate_answer1(qa_item["question"])
        scores.append(rouge_l_score(pred, qa_item["answer"]))

    return sum(scores) / len(scores)

In [115]:
import time

def evaluate_latency(qa_dataset, sample_size=20):
    latencies = []

    for qa_item in qa_dataset[:sample_size]:
        start = time.time()
        generate_answer1(qa_item["question"])
        latencies.append(time.time() - start)

    return {
        "Avg Latency (s)": sum(latencies) / len(latencies),
        "Max Latency (s)": max(latencies)
    }

In [125]:
results = {}

# Answer quality
results.update(evaluate_answer_quality(qa_dataset))
results["ROUGE-L"] = evaluate_rouge_l(qa_dataset)

# Efficiency
results.update(evaluate_latency(qa_dataset))

for k, v in results.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

Exact Match: 0.5900
F1: 0.6403
ROUGE-L: 0.6429
Avg Latency (s): 8.5058
Max Latency (s): 13.0012


In [90]:
!pip install nltk
from nltk.util import ngrams

In [91]:
from sentence_transformers import util

def semantic_similarity(pred, gold):
    emb1 = embedder.encode(pred, convert_to_tensor=True)
    emb2 = embedder.encode(gold, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).item()

def hit_rate_at_k(qa_dataset, k=10):
    return recall_at_k(qa_dataset, k)

import math

def ndcg_at_k(qa_dataset, k=10):
    scores = []

    for qa in qa_dataset:
        urls = get_ranked_urls(qa["question"], k)
        if qa["source_url"] in urls:
            rank = urls.index(qa["source_url"]) + 1
            scores.append(1 / math.log2(rank + 1))
        else:
            scores.append(0)

    return sum(scores) / len(scores)

def contextual_precision(qa_dataset, k=5):
    scores = []

    for qa in qa_dataset:
        urls = get_ranked_urls(qa["question"], k)
        scores.append(1/k if qa["source_url"] in urls else 0)

    return sum(scores)/len(scores)

def contextual_recall(qa_dataset, k=10):
    return recall_at_k(qa_dataset, k) #In URL-grounded QA, contextual recall = retrieval recall.

def grounding_score(answer, context, n=3):
    ans_ngrams = set(ngrams(normalize_text(answer).split(), n))
    ctx_ngrams = set(ngrams(normalize_text(context).split(), n))
    return len(ans_ngrams & ctx_ngrams) / max(len(ans_ngrams), 1)

def hallucination_rate(answer, context):
    if "i don't know" in answer.lower():
        return 0
    return 1 - grounding_score(answer, context)


In [126]:
def full_evaluation(qa_dataset):
    results = {}

    # Retrieval
    results["MRR@10"] = compute_mrr(qa_dataset)
    results["Recall@10"] = recall_at_k(qa_dataset)
    results["Precision@5"] = precision_at_k(qa_dataset)
    results["HitRate@10"] = hit_rate_at_k(qa_dataset)
    results["NDCG@10"] = ndcg_at_k(qa_dataset)
    results["ContextualPrecision@5"] = contextual_precision(qa_dataset)
    results["ContextualRecall@10"] = contextual_recall(qa_dataset)

    # Answer quality (sample-based if slow)
    em, f1, rouge, sem = [], [], [], []

    for qa_item in qa_dataset[:30]: # Renamed loop variable to qa_item
        pred, ctx = generate_answer1(qa_item["question"])
        gold = qa_item["answer"]

        em.append(exact_match(pred, gold))
        f1.append(f1_score(pred, gold))
        rouge.append(rouge_l_score(pred, gold))
        sem.append(semantic_similarity(pred, gold))

    results["EM"] = sum(em)/len(em)
    results["F1"] = sum(f1)/len(f1)
    results["ROUGE-L"] = sum(rouge)/len(rouge)
    results["SemanticSim"] = sum(sem)/len(sem)

    # Efficiency
    results.update(evaluate_latency(qa_dataset))

    return results

In [127]:
results = full_evaluation(qa_dataset)

print("\n=== Evaluation Results ===")
for k, v in results.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")



=== Evaluation Results ===
MRR@10: 0.8087
Recall@10: 0.8900
Precision@5: 0.0890
HitRate@10: 0.8900
NDCG@10: 0.8285
ContextualPrecision@5: 0.1760
ContextualRecall@10: 0.8900
EM: 0.6000
F1: 0.6992
ROUGE-L: 0.6853
SemanticSim: 0.8176
Avg Latency (s): 8.5232
Max Latency (s): 12.8836


In [129]:
import pandas as pd

results = full_evaluation(qa_dataset)

df = pd.DataFrame(results.items(), columns=["Metric", "Score"])
df


,Metric,Score
0,MRR@10,0.808750
1,Recall@10,0.890000
2,Precision@5,0.089000
3,HitRate@10,0.890000
4,NDCG@10,0.828513
5,ContextualPrecision@5,0.176000
6,ContextualRecall@10,0.890000
7,EM,0.600000
8,F1,0.699153
9,ROUGE-L,0.685302


# Innovative Evaluation

In [65]:
def retrieve_dense_only(query, top_k=10):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    q_emb /= np.linalg.norm(q_emb, axis=1, keepdims=True)
    _, ids = faiss_index.search(q_emb.astype("float32"), top_k)
    return [processed_chunks[i]["url"] for i in ids[0]]

def retrieve_sparse_only(query, top_k=10):
    tokens = query.split()
    scores = bm25.get_scores(tokens)
    ids = np.argsort(scores)[::-1][:top_k]
    return [processed_chunks[i]["url"] for i in ids]

def retrieve_hybrid_urls(query, top_k=10):
    return get_ranked_urls(query, top_k)


In [66]:
def ablation_mrr(qa_dataset, mode="hybrid"):
    rr = []
    for qa in qa_dataset:
        if mode == "dense":
            urls = retrieve_dense_only(qa["question"])
        elif mode == "sparse":
            urls = retrieve_sparse_only(qa["question"])
        else:
            urls = retrieve_hybrid_urls(qa["question"])

        rr.append(reciprocal_rank(urls, qa["source_url"]))
    return sum(rr) / len(rr)


In [67]:
print("Dense-only MRR:", ablation_mrr(qa_dataset, "dense"))
print("Sparse-only MRR:", ablation_mrr(qa_dataset, "sparse"))
print("Hybrid MRR:", ablation_mrr(qa_dataset, "hybrid"))


Dense-only MRR: 0.8458730158730158
Sparse-only MRR: 0.4774563492063492
Hybrid MRR: 0.80875


Interpretation

Dense → semantic strength

Sparse → keyword precision

Hybrid → best of both

In [68]:
adversarial_questions = [
    {"q": "What is NOT true about artificial intelligence?", "type": "negated"},
    {"q": "How does photosynthesis affect climate change?", "type": "multi-hop"},
    {"q": "Explain AI in simple terms.", "type": "paraphrase"},
    {"q": "Who invented artificial photosynthesis?", "type": "unanswerable"},
]


In [93]:
def detect_hallucination(answer, context):
    return grounding_score(answer, context) < 0.3


In [96]:
for item in adversarial_questions:
    ans, ctx = generate_answer1(item["q"])
    print(item["type"], "| Hallucination:", detect_hallucination(ans, ctx))

Token indices sequence length is longer than the specified maximum sequence length for this model (905 > 512). Running this sequence through the model will result in indexing errors


negated | Hallucination: True
multi-hop | Hallucination: False
paraphrase | Hallucination: False
unanswerable | Hallucination: False


In [81]:
def categorize_error(qa, predicted_answer, retrieved_urls):
    if qa["source_url"] not in retrieved_urls:
        return "Retrieval Failure"
    elif f1_score(predicted_answer, qa["answer"]) < 0.3:
        return "Generation Failure"
    else:
        return "Context Issue"


In [117]:
from collections import Counter

error_stats = Counter()

for qa_item in qa_dataset:
    pred, _ = generate_answer1(qa_item["question"])
    urls = get_ranked_urls(qa_item["question"])
    err = categorize_error(qa_item, pred, urls)
    error_stats[(qa_item["question_type"], err)] += 1

error_stats

Counter({('factual', 'Context Issue'): 19,
         ('comparative', 'Context Issue'): 20,
         ('inferential', 'Context Issue'): 14,
         ('multi-hop', 'Context Issue'): 9,
         ('comparative', 'Generation Failure'): 5,
         ('multi-hop', 'Retrieval Failure'): 10,
         ('inferential', 'Generation Failure'): 10,
         ('factual', 'Generation Failure'): 6,
         ('inferential', 'Retrieval Failure'): 1,
         ('multi-hop', 'Generation Failure'): 6})

In [118]:
judge_prompt = """
Evaluate the answer on:
1. Factual Accuracy (0–5)
2. Completeness (0–5)
3. Relevance (0–5)
4. Coherence (0–5)

Question:
{question}

Answer:
{answer}

Return scores with brief justification.
"""


In [119]:
def llm_judge(question, answer):
    prompt = judge_prompt.format(question=question, answer=answer)
    return qa(prompt)[0]["generated_text"]


In [120]:
def estimate_confidence(answer, context):
    return grounding_score(answer, context)


In [121]:
import numpy as np

conf, correctness = [], []

for qa_item in qa_dataset[:30]:
    pred, ctx = generate_answer1(qa_item["question"])
    conf.append(estimate_confidence(pred, ctx))
    correctness.append(f1_score(pred, qa_item["answer"]))

np.corrcoef(conf, correctness)[0,1]

np.float64(-0.2739615171637942)

In [122]:
import spacy
nlp = spacy.load("en_core_web_sm")

def entity_coverage(answer, context):
    ans_ents = {e.text for e in nlp(answer).ents}
    ctx_ents = {e.text for e in nlp(context).ents}
    return len(ans_ents & ctx_ents) / max(len(ans_ents), 1)


In [128]:
!pip install reportlab

from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

def generate_pdf_report(results, filename="rag_report.pdf"):
    doc = SimpleDocTemplate(filename)
    styles = getSampleStyleSheet()
    content = []

    content.append(Paragraph("Hybrid RAG Evaluation Report", styles["Title"]))

    for k, v in results.items():
        content.append(Paragraph(f"<b>{k}:</b> {v}", styles["Normal"]))

    doc.build(content)

generate_pdf_report(results)